In [43]:
# import urllib.request
from bs4 import BeautifulSoup
import requests

momo_url = 'http://www.momoshop.com.tw/mosearch/iphone7%20plus%20128g.html'
pchome_url = 'http://ecshweb.pchome.com.tw/search/v3.3/all/results?q=PS4%20CUH&page=1&sort=rnk/dc'

# result = urllib.request.urlopen(pchome_url)

# result = requests.get(url=pchome_url)
# parse_result = BeautifulSoup(result.text, 'lxml')
# html_result = BeautifulSoup(urllib.request.urlopen(pchome_url), 'lxml')
# html_result = BeautifulSoup(requests.get(pchome_url).text, 'lxml')

In [45]:
def get_page(url):
    
    from bs4 import BeautifulSoup
    import urllib.request
    import random
    
    request = urllib.request.Request(url)
    
    ####隨機header挑選####
    foo = [
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36', 
      'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
      'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
      'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_7; da-dk) AppleWebKit/533.21.1 (KHTML, like Gecko) Version/5.0.5 Safari/533.21.1'
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
      'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2722.0 Safari/537.36'
      'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
    ]
    headers = str(random.choice(foo))
    ####隨機header挑選####
    
    request.add_header('User-Agent', headers) 
    response = urllib.request.urlopen(request, timeout=180)
    html = BeautifulSoup(response.read().decode('utf-8'), 'lxml')
    return html


# def get_page2(url):
    
#     from bs4 import BeautifulSoup
#     import urllib.request
#     import random
    
#     request = urllib.request.Request(url)
    
#     ####隨機header挑選####
#     foo = [
#       'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36', 
#       'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
#       'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
#       'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_7; da-dk) AppleWebKit/533.21.1 (KHTML, like Gecko) Version/5.0.5 Safari/533.21.1'
#       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
#       'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2722.0 Safari/537.36'
#       'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
#     ]
# #     headers = str(random.choice(foo))
#     ####隨機header挑選####
    
#     request.add_header('User-Agent', headers={
#             'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
#             'Accept-Encoding':'gzip, deflate, sdch',
#             'Accept-Language:en-US,en;q=0.8,
#             'Connection':'keep-alive'
#         }) 
#     response = urllib.request.urlopen(request, timeout=180)
#     html = BeautifulSoup(response.read().decode('big5'), 'lxml')
#     return html

def momo():
    
    result = get_page(momo_url)
    
    momo = list()
    for i in result.find_all('ul', {'id':'chessboard'}):
        for x in i.find_all(['a', 'b']):
            if x.text != '' and len(momo) != 8:
                momo.append(x.text)
    
    ####將結果變成list裡面包tuple(產品, 價格)####
    momo_display = list()
    a, b = 0, 2
    x = int(len(momo)/2)
    while x != 0:
        momo_display.append(tuple(momo[a:b]))
        x -= 1
        a, b = b, b+2
    ####將結果變成list裡面包tuple(產品, 價格)####
    
    return momo_display



def pchome():
    
    import json
    
    pchome = get_page('http://ecshweb.pchome.com.tw/search/v3.3/all/results?q=iphone7%20plus%20128g&page=1&sort=rnk/dc')
    
    pchome_data = dict()
    for x in pchome.find_all('p'):
        item_data = json.loads(x.text)
        pchome_data = item_data['prods'][:4]
    
    pchome_display = [(i['name'], i['price']) for i in pchome_data]
    
    return pchome_display



def payeasy():
    
    terms = requests.compat.quote('iphone7 128g payeasy') 
    result = get_page('https://www.google.com.tw/search?&q=' + terms + '&oq=' + terms)
    
    payeasy_list = list()
    for url in result.find_all('h3', {'class':'r'}):
        for url2 in url:
            if 'EcCategoryV2' in url2['href']:
                payeasy_list.append(url2['href'])
                
    payeasy_web_content = get_page(payeasy_list[1])
    
    payeasy_result = list()
    for x in payeasy_web_content.find_all('p', {'class':['ProductName', 'ProducPrice']}):
        payeasy_result.append(x.text)
    
    payeasy_display = [x.strip('\r\n').strip(' ').strip('NT$').rstrip('購買') for x in payeasy_result][:8]
    
    ####將結果變成list裡面包tuple(產品, 價格)####
    payeasy_display2 = list()
    a, b = 0, 2
    x = int(len(payeasy_display)/2)
    while x != 0:
        payeasy_display2.append(tuple(payeasy_display[a:b]))
        x -= 1
        a, b = b, b+2
    ####將結果變成list裡面包tuple(產品, 價格)####

    return payeasy_display2

In [297]:
header = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.21 Safari/537.36',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8'
}

req = urllib.request.Request(pchome_url, headers=header)
text = urllib.request.urlopen(req)
BeautifulSoup(response.read().decode('utf-8'), 'lxml')

In [770]:
def get_page_pez(url):
    
    import random
    from bs4 import BeautifulSoup
    import urllib.request
    
    request = urllib.request.Request(url)
    
    ####randon header here####
    foo = [
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36', 
      'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
      'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
      'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_7; da-dk) AppleWebKit/533.21.1 (KHTML, like Gecko) Version/5.0.5 Safari/533.21.1'
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
      'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2722.0 Safari/537.36'
      'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
    ]
    headers = str(random.choice(foo))
    ####randon header here####
    
    request.add_header('User-Agent', headers)
    response = urllib.request.urlopen(request, timeout=180)
    html = BeautifulSoup(response.read().decode('big5'), 'lxml')
    response.close()
    
    return html

# pchome = get_page2('http://ecshweb.pchome.com.tw/search/v3.3/all/results?q=iphone7&page=1')

In [31]:
from bs4 import BeautifulSoup
import urllib.request
# import requests
import random

class price_compare():
    

    def __init__(self, product_name):
        
        self.product_name = product_name ####產品名稱
#         self.display_counts = display_counts ####顯示次數
        self.encode_pro_name = urllib.parse.quote(product_name) ####url encode
        
        self.__momo_url = 'http://www.momoshop.com.tw/mosearch/' + self.encode_pro_name + '.html'
        self.__pchome_url = 'http://ecshweb.pchome.com.tw/search/v3.3/all/results?q=' + self.encode_pro_name + '&page=1&sort=rnk/dc'
    
    
    def get_page(self, url):

        request = urllib.request.Request(url)

        ####隨機header挑選####
        foo = [
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
            'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
            'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_7; da-dk) AppleWebKit/533.21.1 (KHTML, like Gecko) Version/5.0.5 Safari/533.21.1'
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2722.0 Safari/537.36'
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
        ]
        headers = str(random.choice(foo))
        ####隨機header挑選####

        request.add_header('User-Agent', headers)
        response = urllib.request.urlopen(request, timeout=180)
        html = BeautifulSoup(response.read().decode('utf-8'), 'lxml')
        response.close()

        return html
    
    def get_page2(self, url):

        request = urllib.request.Request(url)
        
        ####隨機header挑選####
        foo = [
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
            'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
            'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_7; da-dk) AppleWebKit/533.21.1 (KHTML, li e Gecko) Version/5.0.5 Safari/533.21.1'
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2722.0 Safari/537.36'
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
        ]
        headers = str(random.choice(foo))
        ####隨機header挑選####

        request.add_header('User-Agent', headers)
        response = urllib.request.urlopen(request, timeout=180)
        html = BeautifulSoup(response.read().decode('big5'), 'lxml')
        response.close()

        return html
    

    def momo(self):

        result = self.get_page(self.__momo_url)
        return result

#         momo = list()
        
#         if result.find_all('ul', {'id': 'chessboard'}):
#             for i in result.find_all('ul', {'id': 'chessboard'}):
#                 for x in i.find_all(['a', 'b']):
# #                 if x.text != '' and len(momo) != 8:
#                     if x.text != '':
#                         momo.append(x.text)
            
#         else:
#             for i in result.find_all('script'):
#                 momo_goods_url = i.text.strip('location.href=').strip(';').strip('\'"').replace(',','')
#             momo_goods_detail = self.get_page2(momo_goods_url)
#             for i in momo_goods_detail.find_all('div', {'class':'prdnoteArea'}):
#                 momo.append(i.find('h1').text)
#             for price in i.find('span'):
#                 momo.append(price.replace(',',''))
            
#         ####將結果變成list裡面包tuple(產品, 價格)####
#         momo_display = list()
#         a, b = 0, 2
#         x = int(len(momo) / 2)
#         while x != 0:
#             momo_display.append(tuple(momo[a:b]))
#             x -= 1
#             a, b = b, b + 2
#         ####將結果變成list裡面包tuple(產品, 價格)####

#         return momo_display
    

    def pchome(self):

        import json
        
        pchome = self.get_page(self.__pchome_url)

        pchome_data = dict()
        for x in pchome.find_all('p'):
            item_data = json.loads(x.text)
            pchome_data = item_data['prods']

        pchome_display = [(i['name'], int(i['price'])) for i in pchome_data]

        return pchome_display
    
    
    def payeasy(self):

        terms = self.encode_pro_name + '%20payeasy'
        result = self.get_page('https://www.google.com.tw/search?&q=' + terms + '&oq=' + terms)

        payeasy_EC = list()
        payeasy_PD = list()
        for url in result.find_all('h3', {'class':'r'}):
            for url2 in url:
                try:
                    if 'EcCategoryV2' in url2['href']:
                        payeasy_EC.append(url2['href'])
                    if 'ProductDetail' in url2['href']:
                        payeasy_PD.append(url2['href'])
                except Exception:
                    pass
#         return payeasy_EC, payeasy_PD
                
        payeasy_EC_result = list()   
        payeasy_PD_result = list()  
           #?page=1&sort=2&direction=1#filterTool
        if payeasy_EC:
            payeasy_get_web_page = self.get_page(payeasy_EC[0])
            pages = list()
            for i in payeasy_get_web_page.find_all('div', {'class':'pagination'}):
                for x in i.find_all('span'):
                    pages = [range(1, int(u)+1) for u in x if u.isdigit()]
            for page in pages[0]:
#                 print(payeasy_EC[0] + '?page=' + str(page) + '&sort=2&direction=1#filterTool')
                payeasy_web_content = self.get_page(payeasy_EC[0] + '?page=' + str(page) + '&sort=2&direction=1#filterTool')
                for x in payeasy_web_content.find_all('p', {'class':['ProductName', 'ProducPrice']}):
                    payeasy_EC_result.append(x.text)
        else:
            payeasy_web_content = self.get_page(payeasy_PD[0])
            for x in payeasy_web_content.find_all('div', {'class':'product_info_area'}):
                for pro_name in x.find_all('h1', {'class':'pro_name_cn'}):
                    payeasy_PD_result.append(pro_name.text)
                for price in x.find_all('div', {'class':'price_001'}):
                    payeasy_PD_result.append(price.find_next('b').text)

#         if payeasy_PD:
#             payeasy_web_content = self.get_page(payeasy_PD[0])
#             for x in payeasy_web_content.find_all('div', {'class':'product_info_area'}):
#                 for pro_name in x.find_all('h1', {'class':'pro_name_cn'}):
#                     payeasy_PD_result.append(pro_name.text)
#                 for price in x.find_all('div', {'class':'price_001'}):
#                     payeasy_PD_result.append(price.find_next('b').text)


        payeasy_display = list()
        if payeasy_EC_result:
            payeasy_display = [x.strip('\r\n').strip(' ').strip('NT$').rstrip('購買').replace(',','').strip('購買\n福登入享福利獨享價') for x in payeasy_EC_result]
        if payeasy_PD_result:
            payeasy_display = [x.replace(',','') for x in payeasy_PD_result]


        ####將結果變成list裡面包tuple(產品, 價格)####
        payeasy_display2 = list()
        a, b = 0, 2
        x = int(len(payeasy_display) / 2)
        while x != 0:
            payeasy_display2.append(tuple(payeasy_display[a:b]))
            x -= 1
            a, b = b, b + 2
        ####將結果變成list裡面包tuple(產品, 價格)####

        return payeasy_display2
    
    
    def gohappy(self):

        import json
        import requests
    
    
        def gohappy_pro_pages(search): #### 取得商品頁數

            gohappy_pro_url = 'http://www.gohappy.com.tw/ec2/search?sid=&hotNum=1&search=' + str(requests.utils.quote(search))
            gohappy_url_html = self.get_page(gohappy_pro_url)
        
            parse_time = None
            for pages in gohappy_url_html.find_all('ul', {'class': 'pagenum'}):
                gohappy_page = pages.text.replace('\n', '')
                page_index = gohappy_page.find('共')
                parse_time = int(gohappy_page[page_index + 1 :].replace('頁', ''))

            return parse_time

        gohappy_url = 'http://www.gohappy.com.tw/ec2/searchCate'

        pages = gohappy_pro_pages(self.product_name)
        gohappy_list = list()

        for page in range(0, pages):
            parameters = {'searchs': self.product_name, 'pageno': str(page), 'pagesize': '20', 'orderby': 'close', 'cateLvs': '-1', 'watchtype': '1'}
            r = requests.post(gohappy_url, params = parameters)
            r_json = json.loads(r.text)
            r_bs = BeautifulSoup(r_json.get('data').get('form'), 'lxml')

            for elements in r_bs.find_all('ol', {'class': 'proddata-list'}):
                for product in elements.find_all('li', {'class': 'prodname'}):
                    gohappy_list.append(product.text.replace('\n', ''))
                for price_tag in elements.find_all('li', {'class': 'price-txt'}):
                    for price in price_tag.find_all('strong'):
                        gohappy_list.append(price.text)

        ####將結果變成list裡面包tuple(產品, 價格)####
        gohappy_display = list()
        a, b = 0, 2
        x = int(len(gohappy_list) / 2)
        while x != 0:
            gohappy_display.append(tuple(gohappy_list[a:b]))
            x -= 1
            a, b = b, b + 2
        ####將結果變成list裡面包tuple(產品, 價格)####

        return gohappy_display


    
    def to_ES(self):
        
        from elasticsearch import Elasticsearch
        es = Elasticsearch([{'host': '10.10.110.155', 'port': 9200}])
        
        momo_data = self.momo()
        pchome_data = self.pchome()
        payeasy_data = self.payeasy()
        
        
        ####計算資料的index####
        def counter():
            len_all = len(momo_data + pchome_data + payeasy_data)
            yield from range(len_all)
        counter = counter()
        ####計算資料的index####
        
        ####insert momo data into elasticsearch#####
        momo_dict = dict()

        for i in range(len(momo_data)):
            momo_dict['name'] = momo_data[i][0]
            momo_dict['price'] = int(momo_data[i][1])
            momo_dict['company'] = 'momo'
            es.index(index='product', doc_type='3C', id=next(counter), body=momo_dict)
        ####insert momo data into elasticsearch#####
            
        ####insert pchome data into elasticsearch####     
        pchome_dict = dict()

        for i in range(len(pchome_data)):
            pchome_dict['name'] = pchome_data[i][0]
            pchome_dict['price'] = int(pchome_data[i][1])
            pchome_dict['company'] = 'pchome'
            es.index(index='product', doc_type='3C', id=next(counter), body=pchome_dict)
        ####insert pchome data into elasticsearch####
          
        ####insert payeasy data into elasticsearch####     
        payeasy_dict = dict()

        for i in range(len(payeasy_data)):
            payeasy_dict['name'] = payeasy_data[i][0]
            payeasy_dict['price'] = int(payeasy_data[i][1])
            payeasy_dict['company'] = 'payeasy'
            es.index(index='product', doc_type='3C', id=next(counter), body=payeasy_dict)
        ####insert payeasy data into elasticsearch####

In [52]:
# price = price_compare('KENBOO(男)-後座力 雙撞色夜螢光運動鞋')
# price = price_compare('iphone7 plus 128GB')
# price = price_compare('復原配方洗髮精(250ml)')
# price = price_compare('皮革清潔保養乳')
price = price_compare("ASUS ZenFone 3 (ZE552KL)5.5吋4G LTE雙卡機(4G/64G版)")

In [21]:
price.gohappy()

[('Mizuno PROFESSIONAL 野手硬式手套 1ATGH50207', '4220'),
 ('MIZUNO 少年右打者打擊手套-棒球 壘球 黑紅', '434'),
 ('MIZUNO 右打者用打擊手套-棒球 壘球 左手用 一只入 美津濃 黑灰藍', '455'),
 ('MIZUNO 右打者用打擊手套-棒球 壘球 左手用 一只入 美津濃 黑紅白', '455'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 藍黑白', '871'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 橘白黑', '871'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 黑白', '871'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 灰黑白', '871'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 丈青白', '871'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 白黑', '871'),
 ('MIZUNO 打擊手套-棒球 壘球 一雙入 美津濃 紅黑白', '871'),
 ('MIZUNO 硬式手套-棒球 壘球 美津濃 丈青橘 F', '3307'),
 ('MIZUNO 硬式手套-棒球 壘球 美津濃 橘紅 F', '3307'),
 ('MIZUNO 日製高級保革霜-棒壘球 配件 美津濃 依賣場 F', '680')]

In [33]:
price.payeasy()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbd in position 44: invalid start byte

In [53]:
price.momo()

<!DOCTYPE HTML>
<html lang="zh">
<head>
<link href="/main/favicon.ico" rel="shortcut icon"/>
<link href="/main/favicon.ico" rel="Bookmark"/>
<link href="/main/favicon.ico" rel="icon" type="image/ico"/>
<title>momo購物網</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="64G版),momo購物網" name="Title"/>
<meta content="momo購物網" name="Author"/>
<meta content="momoshop,momo購物網" name="Subject"/>
<meta content="momo購物網64G版)商品價格優惠便宜,種類規格齊全,值得推薦!" name="Description"/>
<meta content="64G版)" name="Keywords"/>
<meta content="Tradition Chinese" name="Language"/>
<meta content="momoshop,momo購物網" name="Abstract"/>
<meta content="momoshop,momo購物網" name="Copyright"/>
<meta content="momoshop,momo購物網" name="Designer"/>
<meta content="momoshop,momo購物網" name="Publisher"/>
<meta content="Global" name="Distribution"/>
<meta content="All" name="Robots"/>
<!-- HTTP 1.1 -->
<meta content="no-cache" http-equiv="Ca

In [46]:
get_page('http://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=4492052&Area=search&mdiv=403')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 312: invalid start byte

In [38]:
for i in aa.find_all('ul', {'class': 'prdPrice'}):
    print(i)

In [181]:
price.pchome()

[('【美津濃MIZUNO】5MJML60409(黑X黑)只 3D STRETCH 高爾夫手套', 380),
 ('【美津濃MIZUNO】330286.9000(黑X黑)VINTAGE PRO雙扣式腕帶棒壘打擊手套(雙)', 780),
 ('【美津濃MIZUNO】330286.1010(深紅X黑)VINTAGE PRO雙扣式腕帶棒壘打擊手套(雙)', 780),
 ('【美津濃MIZUNO】1ETEA61707(灰X藍X黑)服貼剪裁左打者用棒壘打擊手套(只)', 380),
 ('【美津濃MIZUNO】5MJML55427(淺灰X藍) 仿毛皮GOLF 手套', 390),
 ('【美津濃MIZUNO】330286.5252(藍X黑)VINTAGE PRO雙扣式腕帶棒壘打擊手套(雙)', 780),
 ('【美津濃MIZUNO】1ETEA60707(灰X藍X黑)服貼剪裁右打者用棒壘打擊手套(只)', 380),
 ('【美津濃MIZUNO】5MJML55409(淺灰X黑) 仿毛皮GOLF 手套', 390),
 ('【美津濃MIZUNO】330286.8585(迷彩X黑)VINTAGE PRO雙扣式腕帶棒壘打擊手套(雙)', 780),
 ('【美津濃MIZUNO】5MJML55462(淺灰X紅) 仿毛皮GOLF 手套', 390),
 ('【美津濃MIZUNO】1ETEA60716(紅X白X黑)服貼剪裁右打者用棒壘打擊手套(只)', 380),
 ('【美津濃MIZUNO】5MJML60401(白X黑)只    3D STRETCH 高爾夫手套', 380),
 ('【美津濃MIZUNO】330286.2020(橘X黑)VINTAGE PRO雙扣式腕帶棒壘打擊手套(雙)', 780),
 ('【美津濃MIZUNO】1ETEA61716(紅X白X黑)服貼剪裁左打者用棒壘打擊手套(只)', 380),
 ('【美津濃MIZUNO】1ETEA61795(橘X白X黑)服貼剪裁左打者用棒壘打擊手套(只)', 380),
 ('【美津濃MIZUNO】1ETEA60795(橘X白X黑)服貼剪裁右打者用棒壘打擊手套(只)', 380),
 ('Mizuno GOAT HI DURA 打擊手套 1EJEA11309', 888),
 ('Mizuno 美津濃 STARIA 硬式手

In [168]:
price.to_ES()

In [142]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': '10.10.110.155', 'port': 9200}])

In [163]:
es.search(index="product", body={"query": {"match" : { "name" : {"query" : "PS4 CUH 主機", "operator" : "and"}}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '45',
    '_index': 'product',
    '_score': 2.0663652,
    '_source': {'company': 'payeasy',
     'name': '【SONY】PS4 500GB主機 (CUH - 1207)【加：副廠手把果凍套 + 副廠主機直立架】',
     'price': 11450},
    '_type': '3C'},
   {'_id': '17',
    '_index': 'product',
    '_score': 1.999778,
    '_source': {'company': 'pchome',
     'name': 'SONY PS4主機 CUH-2017A系列500GB-極致黑',
     'price': 11680},
    '_type': '3C'},
   {'_id': '31',
    '_index': 'product',
    '_score': 1.999778,
    '_source': {'company': 'payeasy',
     'name': 'SONY PS4主機 CUH-2017系列500GB-極致黑',
     'price': 9980},
    '_type': '3C'},
   {'_id': '37',
    '_index': 'product',
    '_score': 1.7543509,
    '_source': {'company': 'payeasy',
     'name': 'SONY PS4 Slim 500G (CUH-2017系列) 主機 (台灣公司貨)+高質感主機收納包【限量加贈：造型類比套】',
     'price': 10680},
    '_type': '3C'},
   {'_id': '1',
    '_index': 'product',
    '_score': 1.7198174,
    '_source': {'company': 'momo',


In [169]:
es.index(index='product', doc_type='3C', id=50, body= {'name': '【Sam老師 UglyMaker】AVEDA 復原配方洗髮精 (450ml)', 
                                                       'price': 1200, 'company': 'Sam'})

{'_id': '50',
 '_index': 'product',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '3C',
 '_version': 1,
 'created': True,
 'result': 'created'}

In [177]:
es.search(index="product", body={"query": {"match" : {"name" : {"query" : "復原配方洗髮精 450ml", "operator" : "and"}}}})
# es.search(index="product", body={"query":{"match":{"name":{"query":"復原配方洗髮精", "fuzziness": "AUTO","operator":"and"}}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '50',
    '_index': 'product',
    '_score': 14.94088,
    '_source': {'company': 'Sam',
     'name': '【Sam老師 UglyMaker】AVEDA 復原配方洗髮精 (450ml)',
     'price': 1200},
    '_type': '3C'}],
  'max_score': 14.94088,
  'total': 1},
 'timed_out': False,
 'took': 5}

In [ ]:
es.get(index='product', doc_type='3C', id=44)

In [48]:
import requests
res = requests.get('http://10.10.110.155:9200/')
print(res.content.decode('utf-8'))

{
  "name" : "cqfAD8U",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "vTKUvCLvQJ-X3DFf00rUow",
  "version" : {
    "number" : "5.1.1",
    "build_hash" : "5395e21",
    "build_date" : "2016-12-06T12:36:15.409Z",
    "build_snapshot" : false,
    "lucene_version" : "6.3.0"
  },
  "tagline" : "You Know, for Search"
}



In [49]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '10.10.110.155', 'port': 9200}])

In [929]:
# es_result = es.search(index="product", body={"query":{"match":{"name":{"query":"iphone 7 128g","fuzziness": "AUTO","operator":"and"}}}})
es_result = es.search(index="product", body={"size" : 1000, "query":{"match":{"name":"iphone 7 plus 128g"}}})

In [ ]:
es_result['hits']

In [733]:
pez = get_page2('http://www.payeasy.com.tw/search/search3/index.html?q=iphone&lv=2')

In [747]:
test = urllib.request.urlopen('http://www.payeasy.com.tw/search/search3/index.html?q=mizuno&lv=2')

In [773]:
haha = urllib.request.urlopen('http://www.payeasy.com.tw/search/search3/index.html?q=mizuno&lv=2')

In [815]:
headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding':'gzip, deflate, sdch',
    'Accept-Language':'en-US,en;q=0.8',
    'Cache-Control':'max-age=0',
    'Connection':'keep-alive',
    'Host':'www.payeasy.com.tw',
    'If-Modified-Since':'Tue, 03 May 2016 06:01:44 GMT',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.28 Safari/537.36'
}
data = {"pidNum":"4752388","webNum":"6351"}

ppp = requests.post('http://member.payeasy.com.tw/cart/REST/GetWelfareProducts?Params=%5B%7B%22pidNum%22%3A%224752388%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752330%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224263763%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752348%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752234%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224725028%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752382%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752179%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752314%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224702346%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224725049%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224367873%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752163%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752202%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%224752354%22%2C%22webNum%22%3A%226351%22%7D%2C%7B%22pidNum%22%3A%223557623%22%2C%22webNum%22%3A%226771%22%7D%2C%7B%22pidNum%22%3A%223557705%22%2C%22webNum%22%3A%226771%22%7D%2C%7B%22pidNum%22%3A%223557700%22%2C%22webNum%22%3A%226771%22%7D%2C%7B%22pidNum%22%3A%223558051%22%2C%22webNum%22%3A%226771%22%7D%2C%7B%22pidNum%22%3A%223557710%22%2C%22webNum%22%3A%226771%22%7D%5D')

In [816]:
urllib.parse.u

{"code":"OK","items":[{"cartMaster":{"scmNum":0,"memNum":0,"productType":"Single","modifyDate":"\/Date(-62135596800000+0800)\/","prepareDay":0,"actRealPayment":0.0,"webNum":0,"coupons":[]},"cartDetails":[{"scdNum":0,"pidNum":4752388,"pidQty":0,"pidPromoDesc":"","proMvalue":7500.0,"pidName":"MIZUNO眼鏡 β 鈦休閒/黑-銀 #MF1508 C12","pidSvalue":4500.0,"pidPicPath1":"//images.payeasy.com.tw/products/4777/4749/4749737.JPG","pidUrl":"http://www.payeasy.com.tw/ECShop/ProductDetail.jsp?pidnum=4752388","maxSaleQty":0,"valid":false,"bestbuyProducts":[],"proWeigh":0.0,"isShowWelfareOnlyPrice":false,"welfareOnlyPrice":0.0},{"scdNum":0,"pidNum":4752330,"pidQty":0,"pidPromoDesc":"","proMvalue":7500.0,"pidName":"MIZUNO眼鏡 β 鈦簡約/黑 #MF1404 C51","pidSvalue":4500.0,"pidPicPath1":"//images.payeasy.com.tw/products/4777/4749/4749694.JPG","pidUrl":"http://www.payeasy.com.tw/ECShop/ProductDetail.jsp?pidnum=4752330","maxSaleQty":0,"valid":false,"bestbuyProducts":[],"proWeigh":0.0,"isShowWelfareOnlyPrice":false,"welfare

In [160]:
import requests

pchome = requests.get('http://ecshweb.pchome.com.tw/search/v3.3/?q=ps4%20cuh&scope=all')
pchome_html = BeautifulSoup(pchome.text, 'lxml')

In [161]:
pchome_html

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="zh-tw" http-equiv="Content-Language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="CPfgp1JzxusG1iSc" name="baidu-site-verification"/>
<title>PChome ç·ä¸è³¼ç© - ååæå°</title>
<link href="/opensearch_shopping.xml" rel="search" title="PChome ç·ä¸è³¼ç© ååæå°" type="application/opensearchdescription+xml"/>
<link href="//shopping.pchome.com.tw/min/g=ecshweb-shv3.3-cssv2&amp;201612091230" rel="stylesheet" type="text/css"/>
<script src="/min/g=v3.3shjs&amp;20161207" type="text/javascript"></script>
</head>
<body>
<div class="ecsite-layout style_shopping ecsite-search" id="WRAPPER">
<!-- HEADER start -->
<div id="HEADER">
<div class="block_H">
<div class="Ht">
<h1 class="logotype"><a href="http://

In [162]:
print('\u300a\u65b0\u6a5f\u4e0a\u5e02\u300b\u25bc4K\u756b\u8cea\u5a1b\u6a02\u9ad4\u9a57PlayStation')

《新機上市》▼4K畫質娛樂體驗PlayStation
